# Generating music with Markov Chains using real music as training data

## Import All Required Libraries

In [1]:
import pretty_midi as pm
from collections import defaultdict
import random
from IPython.display import Audio
import numpy as np
from scipy.io.wavfile import write

## Extract Notes from a MIDI File

In [2]:
def extract_notes_from_midi(midi_file_path):
    # List to store all extracted notes from the MIDI file
    extracted_notes = []
    
    # Load the MIDI file
    midi_data = pm.PrettyMIDI(midi_file_path)
    
    # Extract notes from all instruments
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            extracted_notes.append(pm.note_number_to_name(note.pitch))
    
    # Return the extracted notes
    return extracted_notes

In [3]:
# MIDI file path
midi_path = "midi_files/Never-Gonna-Give-You-Up-2.mid"

# Calling the extract_notes_from_midi() function and storing all the extracted notes
notes = extract_notes_from_midi(midi_path)

## Build Transformation Matrix

In [4]:
def build_transition_matrix(music_notes):
    # Dictionary to store lists of possible next note, to the current note (key)
    transition_matrix_new = defaultdict(list)
    
    # Iterating through the list of notes.
    # Each subsequent note is appended to the list of the current note in the matrix.
    for i in range(len(music_notes) - 1):
        current_note = music_notes[i]
        next_note = music_notes[i + 1]
        
        transition_matrix_new[current_note].append(next_note)
    
    # Return the transition matrix
    return transition_matrix_new

In [5]:
transition_matrix = build_transition_matrix(notes)

## Generate Music 

In [6]:
def generate_music(matrix, start_note, length=50):
    # List to store all generated notes
    music = [start_note]
    # The current note
    current_note = start_note
    
    # Iterating through the list of notes
    for _ in range(length - 1):
        # The next note is chosen at random
        next_note = random.choice(matrix[current_note])
        # The next note is added to the `music` list
        music.append(next_note)
        # The current note is updated
        current_note = next_note
    
    # Return all the generated notes
    return music

In [7]:
# Starting note is chosen at random
starting_note = random.choice(notes)
# Generate the music
generated_music = generate_music(transition_matrix, starting_note, length=20)

## Save Generated Music

In [8]:
def save_generated_music_as_wav(gen_music, file_path, sample_rate=44100):
    # Create a PrettyMIDI object
    midi_data = pm.PrettyMIDI()
    # Create an Instrument instance
    instrument = pm.Instrument(program=0)  # program=0 is Acoustic Grand Piano
    
    # Set up the start time and the duration for the note
    start_time = 0.0
    duration = 0.5
    
    # Add notes to the instrument
    for note_name in gen_music:
        note_number = pm.note_name_to_number(note_name)
        note = pm.Note(velocity=50,   # Volume Level
                                pitch=note_number, 
                                start=start_time, 
                                end=start_time + duration)
        instrument.notes.append(note)
        
        start_time += duration
    
    # Add the instrument to the PrettyMIDI object
    midi_data.instruments.append(instrument)
    
    # Synthesize the MIDI data directly to audio (wav)
    audio_data = midi_data.fluidsynth(fs=sample_rate)
    
    # Normalize audio to 16-bit PCM range
    audio_data = np.int16(audio_data / np.max(np.abs(audio_data)) * 32767)
    
    # Write the audio data to a .wav file
    write(file_path, sample_rate, audio_data)

In [9]:
# Set the .wav file name
wav_filename = "markov_music_inspired.wav"

# Save the generated music as MIDI file
save_music = save_generated_music_as_wav(generated_music, wav_filename)

## Play the Generated Music

In [10]:
# Play the generated audio file
audio_path = wav_filename
Audio(audio_path)

## Credits

1. MIDI Files Downloaded from [BitMidi](https://bitmidi.com)